## Overview
#### Arbitrage is the practice of taking advantage of price differences in different markets to make a profit. In the context of options trading, arbitrage is used to exploit price discrepancies between options and their underlying assets. Traders can use arbitrage strategies to profit from mispricings in options by simultaneously buying and selling related assets to lock in a risk-free profit. 

#### When identifying which option to engage in for arbitrage, traders typically look for pricing discrepancies between the options market and the underlying asset market. They may compare the prices of options with different strike prices, expiration dates, or underlying assets to find opportunities for arbitrage. By conducting thorough analysis and monitoring market conditions, traders can identify and execute profitable arbitrage trades in the options market.

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from arch import arch_model
from datetime import datetime, timedelta

In [3]:
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price





def black_scholes_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price





def get_option_data(ticker):
    stock = yf.Ticker(ticker)
    expirations = stock.options
    option_data = pd.DataFrame()
    for expiration in expirations:
        option_chain = stock.option_chain(expiration)
        calls=option_chain.calls
        calls["Option Type"] = "Call"
        puts = option_chain.puts
        puts["Option Type"] = "Put"  
        options = pd.concat([calls, puts])
        options["Expiration Date"] = pd.to_datetime(expiration) + pd.DateOffset(hours =23, minutes = 59, seconds=59)
        option_data = pd.concat([option_data,options])
    calls = option_data[option_data["Option Type"] == "Call"]
    puts = option_data[option_data["Option Type"] == "Put"]
    return calls, puts





def calculate_historical_volatility(ticker, period='20y'):
    df = yf.Ticker(ticker).history(period=period)
    df['returns'] = np.log(df['Close'] / df['Close'].shift(1))
    volatility = df['returns'].std() * np.sqrt(252) 
    return df, volatility





def train_volatility_model(df):
    df['returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['volatility'] = df['returns'].rolling(window=21).std() * np.sqrt(252)
    df = df.dropna()
    for lag in range(1, 22):
        df[f'return_lag_{lag}'] = df['returns'].shift(lag)
    df = df.dropna()
    X = df[[f'return_lag_{lag}' for lag in range(1, 22)]]
    y = df['volatility']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
    return model, df





def predict_volatility(model, df):
    recent_data = df.tail(1)
    X_new = recent_data[[f'return_lag_{lag}' for lag in range(1, 22)]].values
    predicted_volatility = model.predict(X_new.reshape(1, -1))[0]
    return predicted_volatility





def calculate_arbitrage_opportunities(calls, puts, S, r, sigma):
    arbitrage_opportunities = []
    for index, row in calls.iterrows():
        K = row['strike']
        T = (pd.to_datetime(row['Expiration Date']) - pd.Timestamp.today()).days / 365.0
        market_price = row['lastPrice']
        theoretical_price = black_scholes_call(S, K, T, r, sigma)
        if theoretical_price > market_price:
            arbitrage_opportunities.append(('call', K, market_price, theoretical_price, row['Expiration Date'], row['lastTradeDate']))
    for index, row in puts.iterrows():
        K = row['strike']
        T = (pd.to_datetime(row['Expiration Date']) - pd.Timestamp.today()).days / 365.0
        market_price = row['lastPrice']
        theoretical_price = black_scholes_put(S, K, T, r, sigma)
        if theoretical_price > market_price:
            arbitrage_opportunities.append(('put', K, market_price, theoretical_price, row['Expiration Date'], row['lastTradeDate']))
    return arbitrage_opportunities





def main( ticker):
    r = 0.01 
    df, historical_volatility = calculate_historical_volatility(ticker)
    model, df_1 = train_volatility_model(df)
    sigma = predict_volatility(model, df_1)
    S = yf.Ticker(ticker).history(period='30d')['Close'][0] 
    calls, puts = get_option_data(ticker)
    arbitrage_opportunities = calculate_arbitrage_opportunities(calls, puts, S, r, sigma)
    options_list = list()
    options_list_rel = list()
    for opp in arbitrage_opportunities:
        option_type, strike, market_price, theoretical_price, expiration_date, last_trade_date = opp
        if last_trade_date>=(pd.Timestamp(datetime.now(), tz='UTC') - timedelta(days=3)) and (expiration_date<=(pd.Timestamp(datetime.now()) + timedelta(days=300))):
            '''
            print(f"Arbitrage Opportunity: {option_type.upper()} at strike {strike}")
            print(f"Market Price: {market_price}, Theoretical Price: {theoretical_price}")
            print(f"Potential Profit: {theoretical_price - market_price}")
            print(f"Expiration date: {expiration_date}, Last Trade Date: {last_trade_date}")
            print("--------")
            '''
            options_list.append(((theoretical_price - market_price),option_type, strike, market_price, theoretical_price, expiration_date, last_trade_date, ticker))
            options_list_rel.append(((theoretical_price - market_price)*100/market_price,option_type, strike, market_price, theoretical_price, expiration_date, last_trade_date, ticker))
        else:
            pass
    max_value = max(options_list, key=lambda x: x[0])
    max_value_rel = max(options_list_rel, key=lambda x: x[0])
    return max_value, max_value_rel





def optimal_profit(tickers):
    results_list = list()
    for ticker in tickers:
        max_abs, max_rel = main(ticker)
        results_list.append((max_abs, max_rel, ticker)) 
    final_list_abs = list()
    final_list_rel = list()
    for i in range(len(results_list)):
        final_list_abs.append((results_list[i][0][0], results_list[i][0]))
        final_list_rel.append((results_list[i][1][0], results_list[i][1]))
    max_value = max(final_list_abs, key=lambda x: x[0])
    max_value_rel = max(final_list_rel, key=lambda x: x[0])     
    return max_value, max_value_rel

In [4]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'TSLA', 'JPM', 'JNJ',
            'V', 'WMT', 'PG', 'MA', 'NVDA', 'HD', 'DIS', 'BAC', 'UNH', 'VZ', 'PFE',
              'INTC', 'T', 'MRK', 'CRM', 'KO', 'CMCSA', 'NFLX', 'XOM', 'ABT', 'NKE', 
              'TMO', 'ADBE', 'CSCO', 'PYPL', 'MCD', 'IBM', 'QCOM', 'COST', 'AVGO', 'ACN', 
              'TXN', 'HON', 'ORCL', 'LIN', 'UPS', 'LLY', 'SBUX', 'AMD', 'GILD', 'CHTR', 'BKNG',
                'COP', 'MO', 'CAT', 'FIS', 'C', 'CME', 'DHR', 'SPGI', 'LOW', 'FDX', 'NEE', 'NOW', 
                'ISRG', 'CVS', 'RTX', 'TMUS', 'TM', 'AMT', 'EL', 'ADP', 'APD', 'INTU', 'LMT', 'MU', 'ZTS', 
                'ECL', 'GM', 'WBA', 'MMC', 'TFC', 'VRTX', 'COF', 'ICE', 'MCO', 'ADI', 'DD', 'LRCX', 
                'REGN', 'SO', 'MET']



max_abs, max_rel = optimal_profit(tickers)

C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'return_lag_{lag}'] = df['returns'].shift(lag)
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'return_lag_{lag}'] = df['returns'].shift(lag)
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

Mean Squared Error: 0.01912974080123266


c:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'return_lag_{lag}'] = df['returns'].shift(lag)
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

Mean Squared Error: 0.017909646903962927


C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'return_lag_{lag}'] = df['returns'].shift(lag)
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'return_lag_{lag}'] = df['returns'].shift(lag)
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

...

c:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:31: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  options = pd.concat([calls, puts])
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'return_lag_{lag}'] = d

Mean Squared Error: 0.01299429773797272


c:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:31: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  options = pd.concat([calls, puts])
C:\Users\a-spyrosploussiou\AppData\Local\Temp\7\ipykernel_25828\2651168964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'return_lag_{lag}'] = d

Mean Squared Error: 0.07472637573978726


c:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [5]:
columns_abs = ['Absolute value increase','option_type', 'strike', 'market_price', 'theoretical_price', 'expiration_date', 'last_trade_date', 'ticker']
abs_max=pd.DataFrame([max_abs[1]], columns = columns_abs)
abs_max

,Absolute value increase,option_type,strike,market_price,theoretical_price,expiration_date,last_trade_date,ticker
0,457.079759,put,1900.0,164.55,621.629759,2024-06-21 23:59:59,2024-06-14 19:27:09+00:00,AVGO


In [6]:
columns_rel = ['Relative value increase %','option_type', 'strike', 'market_price', 'theoretical_price', 'expiration_date', 'last_trade_date', 'ticker']
rel_max=pd.DataFrame([max_rel[1]], columns = columns_rel)
rel_max

,Relative value increase %,option_type,strike,market_price,theoretical_price,expiration_date,last_trade_date,ticker
0,293410.25799,put,1425.0,0.05,146.755129,2024-06-21 23:59:59,2024-06-14 17:58:26+00:00,AVGO
